In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import os 
from preprocessing import preprocessing, create_sequences
from model.LSTMAutoEncoder import LSTMAutoEncoder
from training import training_loop

%reload_ext autoreload
%autoreload 2

In [3]:
filepath = os.path.join(".", "data", "YaoBox", "Box2StrainRaw1.csv")
t, value = preprocessing(filepath)

train_size = .6
val_size = .2
test_size = .2
train_data = value[:int(len(value) * train_size)]
val_data = value[int(len(value) * train_size):int(len(value) * (train_size + val_size))]
test_data = value[int(len(value) * (train_size + val_size)):]

window_size = 50
train_sequences = create_sequences(train_data, window_size)
val_sequences = create_sequences(val_data, window_size)
test_sequences = create_sequences(test_data, window_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = torch.tensor(train_sequences, dtype=torch.float32).unsqueeze(-1).to(device)
val_dataset = torch.tensor(val_sequences, dtype=torch.float32).unsqueeze(-1).to(device)
test_dataset = torch.tensor(test_sequences, dtype=torch.float32).unsqueeze(-1).to(device)

print(train_dataset.shape, val_dataset.shape, test_dataset.shape)

torch.Size([20417, 50, 1]) torch.Size([6772, 50, 1]) torch.Size([6773, 50, 1])


In [6]:
model = LSTMAutoEncoder().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
model

LSTMAutoEncoder(
  (encoder): Encoder(
    (lstm1): LSTM(1, 64, batch_first=True)
    (lstm2): LSTM(64, 16, batch_first=True)
  )
  (decoder): Decoder(
    (lstm1): LSTM(16, 64, batch_first=True)
    (lstm2): LSTM(64, 1, batch_first=True)
    (fc): Linear(in_features=1, out_features=1, bias=True)
  )
)

In [7]:
train_loss, val_loss, accuracy = training_loop(model, train_dataset, val_dataset, optimizer, loss_fn, epochs=100)


Epoch 0: Train Loss: 0.9527671933174133, Val Loss: 0.7287243008613586
Epoch 10: Train Loss: 0.64486163854599, Val Loss: 0.4779908061027527
Epoch 20: Train Loss: 0.5626871585845947, Val Loss: 0.4225757420063019
Epoch 30: Train Loss: 0.5464721918106079, Val Loss: 0.40932711958885193
Epoch 40: Train Loss: 0.5325376391410828, Val Loss: 0.39770573377609253
Epoch 50: Train Loss: 0.5192403793334961, Val Loss: 0.386606901884079
Epoch 60: Train Loss: 0.5062806606292725, Val Loss: 0.375809907913208
Epoch 70: Train Loss: 0.4935781955718994, Val Loss: 0.3652551770210266
Epoch 80: Train Loss: 0.4811131954193115, Val Loss: 0.3549274802207947
Epoch 90: Train Loss: 0.4688847064971924, Val Loss: 0.3448256552219391


<bound method Module.parameters of LSTMAutoEncoder(
  (encoder): Encoder(
    (lstm1): LSTM(1, 64, batch_first=True)
    (lstm2): LSTM(64, 16, batch_first=True)
  )
  (decoder): Decoder(
    (lstm1): LSTM(16, 64, batch_first=True)
    (lstm2): LSTM(64, 1, batch_first=True)
    (fc): Linear(in_features=1, out_features=1, bias=True)
  )
)>